# Load Libraries

In [1]:
import xml.etree.ElementTree as ET
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from scipy.stats import pearsonr
from sklearn import preprocessing

import zipfile
from glob import glob

import seaborn as sns
from sympy import subsets
sns.set()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

#import japanize_matplotlib
import unicodedata
import mojimoji

from MyZipClass import MyZipClass
from NIKKEIHokenClass import NIKKEIHokenClass
from MyKanjyaClass import MyKanjyaClass
from RECEPTHokenClass import RECEPTHokenClass

from MyMasterClass import MyMasterClass


## Main Class

In [3]:

class RECEPTHokenCheckClass(MyMasterClass):

    def __init__(self):

        self.data_dir = u"S:\\"
        self.output_dir = u"L:\\epson_pxm840f\\Shikakku"

        self.shikaku_tag = {}        
        self.shikaku = []

        # expand latest shikaku history file
        zipObject = MyZipClass()
        self.df_shikaku = zipObject.readZipFile()

        # change pd_datetime to merge recept class
        self.df_shikaku.birth = pd.to_datetime( self.df_shikaku.birth )    
        #zipObject.toCsv(self.df_shikaku,"資格確認_保険証.csv")
        #print("shikaku file size : ", self.df_shikaku.shape)

        # read NIKKEI data to hold Domestic/Social Insurance
        receptObj = RECEPTHokenClass()
        df = receptObj.integrateInsurancedata()
        self.df_recept = df[ ((df.kouhi1.str[:2] != "12") & (df.kouhi1.str[:2] != "15")) ].copy()
        print("RECEPT file size : ", self.df_recept.shape)

    def dftoCsv(self,df, filename):
        super().toCsv(df, filename)
        #filename = os.path.join(self.output_dir,filename)
        #df.to_csv(filename, index=False, encoding='cp932', errors='replace')

    def mergeCheck(self):
        
        selected=['chozai','id','Name','birth','InsurerSegment','InsurerNumber',
                    'InsuredCardSymbol','InsuredIdentificationNumber',
                    'kouhi1','kouhi_jyu1','kouhi2','kouhi_jyu2',
                    'kyufu','institution']

    
        df_merged = pd.merge(self.df_recept, self.df_shikaku, on=["birth"], how="left")

        masks = (df_merged.InsurerNumber_x == df_merged.InsurerNumber_y)  &  \
            (df_merged.InsuredCardSymbol_x == df_merged.InsuredCardSymbol_y )  &   \
             (df_merged.InsuredIdentificationNumber_x == df_merged.InsuredIdentificationNumber_y) & \
                 (df_merged.sex == df_merged.sex1)

        print("** NON confirmed file output shikaku with RECEPT file .....")
        #print(df_merged[masks])
        return df_merged
        #self.toCsv(df_merged[~masks],"レセプト_byBirth.csv")



In [5]:
obj = RECEPTHokenCheckClass()
df_merged = obj.mergeCheck()
df_merged.fillna("", inplace=True)

['ÄæèièmöFùÜù≡_20220224143849389.csv']
change current directory --> L:\epson_pxm840f\Shikakku
## delte unzipped shikaku file... L:\epson_pxm840f\Shikakku\資格確認履歴_20220224143849389.csv
[ReceiptyClass] recepty csv data file size -->  (6019, 180)
[ReceiptyClass] recepty csv data file size -->  (2473, 180)
RECEPT file size :  (347, 19)
** NON confirmed file output shikaku with RECEPT file .....


In [6]:
masks = (df_merged.InsurerNumber_x.apply(str) == df_merged.InsurerNumber_y)  &  \
            (df_merged.InsuredCardSymbol_x == df_merged.InsuredCardSymbol_y )  &   \
             (df_merged.InsuredIdentificationNumber_x == df_merged.InsuredIdentificationNumber_y) # & \
                # (df_merged.sex == df_merged.sex1)


In [7]:
sum(masks)

112

In [9]:
df_tmp = df_merged[~masks]
sex_mask = df_tmp.sex == df_tmp.sex1 

In [18]:
obj.toCsv(df_tmp,"recept_notmatched.csv")

In [16]:
df_tmp.shape

(236, 50)

In [17]:
sum(sex_mask)

30

In [11]:
df_tmp_unmatched = df_tmp[~sex_mask]

In [ ]:
display(df_tmp_unmatched)

In [15]:
df_tmp_unmatched.InsurerNumber_y == ""

0      True
1      True
5      True
11     True
12     True
       ... 
314    True
315    True
316    True
317    True
318    True
Name: InsurerNumber_y, Length: 206, dtype: bool